In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import dotenv
dotenv.load_dotenv()

In [ ]:
df_london_smartmeter = pd.read_csv("../data/LondonSmartMeter/london_smart_meters_dataset_without_missing_values_first_30_consumers.csv", index_col='Time', parse_dates=['Time'])
df_kddcup = pd.read_csv("../data/KDDCup_2018/kdd_cup_2018_dataset_without_missing_values.csv", index_col='Time', parse_dates=['Time'])
df_electricity_321_hourly = pd.read_csv("../data/Electricity321Hourly/electricity_hourly_dataset.csv", index_col='Time', parse_dates=['Time'])
df_electricity_370 = pd.read_csv("../data/Electricity370/LD2011_2014_first_40_consumers.csv", index_col='Time', parse_dates=['Time'])


In [ ]:
import torch
import psutil
import pynvml

# Check if CUDA is available
if torch.cuda.is_available():
    pynvml.nvmlInit()
    # Get the current device
    device = 0
    for i in range(torch.cuda.device_count()):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
        print(
            f"device {i}: {torch.cuda.get_device_properties(i).name} | {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB | Utilization: {utilization.gpu}%"
        )
        device = i

    pynvml.nvmlShutdown()
    print(f"Using CUDA device {device}: {torch.cuda.get_device_name(device)}")
else:
    device = "cpu"
    print("CUDA is not available")

print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", total_cores := psutil.cpu_count(logical=True))

core_percentages = [f"{i}:\t {percentage}%\t" for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1))]
core_percentages_formatted = [core_percentages[i : i + 8] for i in range(0, len(core_percentages), 8)]
for core_group in core_percentages_formatted:
    print(" ".join(core_group))

# RAM Information
svmem = psutil.virtual_memory()
print(f"Total: {svmem.total / (1024**3):.2f} GB")
print(f"Available: {svmem.available / (1024**3):.2f} GB")
print(f"Used: {svmem.used / (1024**3):.2f} GB")
print(f"Percentage: {svmem.percent}%")

print("Distributed PyTorch available:", torch.distributed.is_available())

In [ ]:
from copy import deepcopy
from ts_inverse.utils import grid_search_params
from ts_inverse.models import FCN_Predictor, CNN_Predictor, GRU_Predictor, TCN_Predictor
from ts_inverse.workers import ForecastingWorker

def start_multi_process(g_config, d_config, m_config, pool_size):
    search_args = []
    search_configs = list(grid_search_params(g_config))
    search_dataset_settings = list(grid_search_params(d_config))
    search_model_settings = list(grid_search_params(m_config))
    for g_config in search_configs:
        for m_config in search_model_settings:
            for d_config in search_dataset_settings:
                fa_models_config = {
                    'features': [
                        # [0, 2, 4], [0, 1, 2], [0, 2]
                        # [0], [1], [2], [3], [4], [5],
                        # [0, 1], [0, 2], [0, 3], [0, 4], [0, 5],
                        # [0, 2, 1], [0, 2, 3], [0, 2, 4], [0, 2, 5],
                        # [0, 1, 2, 3], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 5]
                        # [0, 1, 2]
                        [0],
                    ],
                    'input_size': d_config['observation_days'], # This updated after the dataset is created
                    'output_size': d_config['future_days'], # This is updated after the dataset is created
                }
                search_for_all_models_settings = list(grid_search_params(fa_models_config))
                for fa_models_config in search_for_all_models_settings:
                    g_config['run_number'] = len(search_args)
                    args = (g_config, d_config, m_config, fa_models_config)
                    search_args.append(deepcopy(args))

    print(f"Starting {len(search_args)} processes")
    if pool_size == 1:
        for i, args in enumerate(search_args):
            ForecastingWorker(args[0]['run_number']).worker_process(*args)

global_config = {
    'logger_service': 'wandb',
    'experiment_name': 'forecasting_experiment_3-3-2025',
    'seed': [10],
    'early_stopping_patience': 1000,
    'early_stopping_min_delta': 0.001,
    'num_epochs': 100,
    'batch_size': 1,
    'learning_rate': 0.001,
    'device': 1,
    'verbose': False,
    'pool_size': 1,
    'run_number': -1
}

dataset_config = [
    {
        'dataset': 'electricity_370',
        'columns': [df_electricity_370.columns.tolist()[4:5]],
        'train_stride': 24,
        'observation_days': 1,
        'future_days': 1,
        'normalize': 'minmax',
    },

    # {
    #     'dataset': 'electricity_370',
    #     'columns': [df_electricity_370.columns.tolist()[4:5], df_electricity_370.columns.tolist()[4:10]],
    #     'train_stride': [24, 4, 1],
    #     'validation_stride': 1,
    #     'observation_days': [1, 7, 14, 28],
    #     'future_days': 1,
    #     'normalize': 'minmax',
    # },
    # {
    #     'dataset': 'london_smartmeter',
    #     'columns': [df_london_smartmeter.columns.tolist()[:1]],
    #     'train_stride': 24,
    #     'observation_days': 1,
    #     'future_days': 1,
    #     'normalize': 'minmax',
    # },
    # {
    #     'dataset': 'kddcup',
    #     'columns': [df_kddcup.columns.tolist()[:1]],
    #     'train_stride': 24,
    #     'observation_days': 5,
    #     'future_days': 2,
    #     'normalize': 'minmax',
    # },
]

model_config = [
    # {
    #     '_model': TCN_Predictor,
    #     'hidden_size': 64,
    #     'num_levels': 0,
    #     'kernel_size': 4,
    #     'dilation_factor': 2,
    #     'activation': 'relu',
    #     'use_weight_norm': True,
    #     'init_weights': True
    # },
    {
        '_model': FCN_Predictor,
        'hidden_size': 64,
    }, 
    # {
    #     '_model': CNN_Predictor,
    #     'hidden_size': 64,
    # },
    # {
    #     '_model': GRU_Predictor,
    #     'hidden_size': 64,
    # }
]
start_multi_process(global_config, dataset_config, model_config, pool_size=global_config['pool_size'])